# PAA -  Greed 

## Huffman InteractiveTutorial

A compressão de dados é uma necessidade frequente no dia a dia de qualquer usuário de dispositivos computacionais. Isso porque memória pode ser um recurso custoso, por isso vale a pena quando possível guardar os arquivos em um formato comprimido, tendo em vista otimizar o uso da memoria.

Em termos básicos o algoritmo de Huffman é um algoritmo ambicioso que consiste em uma técnica de compressão sem perda de dados. Ele funciona atribuindo códigos de tamanho variável para cada um dos caractéres a serem codificados, dando os códigos de menor tamanho para os caracteres que mais se repetem, de forma a minimizar o tamanho do código final gerado.


Nesse tutorial interativo, você poderá aprender o passo a passo do algoritmo enquanto utiliza ele para comprimir um arquivo de texto.

# PT 0. Bibliotecas utilizadas no código

In [1]:
import heapq
import os
import filecmp
from IPython.display import clear_output, Image, HTML

# PT 1. Codificação

### 1.1 Estrutura inicial
O primeiro passo é definir a estrutura da nossa heap, que será utilizada para guardar cada um dos nosso caracteres em conjunto com sua frequência de ocorrência e nós a esquerda e a direita na árvore. 

In [2]:
class HeapNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    
    #Funções para comparação entre nodes
    def __lt__(self, other):
        return self.freq < other.freq

    def __eq__(self, other):
        if(other == None):
            return False
        if(not isinstance(other, HeapNode)):
            return False
        return self.freq == other.freq

### 1.2 Arquivo a ser comprimido 
Agora podemos pedir ao usuário o caminho do arquivo a ser comprimido e ler seu conteúdo.

In [3]:
example_files = [file for file in os.listdir('arquivos_exemplo')if not file.startswith('.') and file.endswith('txt') ]

while True:
    print("Digite o caminho do arquivo a ser comprimido ou escolha algum dos exemplos já disponíveis no repositório")
    print("\nArquivos disponíveis:\n")
    for file in example_files:
        print(f'\t{file}')
    
    original_path = input()
    if original_path in example_files:
        original_path = f'arquivos_exemplo/{original_path}'
    
    if os.path.isfile(original_path):
        break
    clear_output()
    print("Arquivo não encontrado, tente novamente.")
    
print(f'\nArquivo escolhido: {original_path}\n')
original_filename, original_file_extension = os.path.splitext(original_path)
with open(original_path, 'r+') as file:
    text = file.read()
    text = text.rstrip()

Digite o caminho do arquivo a ser comprimido ou escolha algum dos exemplos já disponíveis no repositório

Arquivos disponíveis:

	musica_vai_lembrar_de_mim.txt
musica_vai_lembrar_de_mim.txt

Arquivo escolhido: arquivos_exemplo/musica_vai_lembrar_de_mim.txt



### 1.3 Contagem de frequência 

Para executar o algoritmo de Huffman temos que primeiro contabilizar as frequências de ocorrência de cada caracter. Aqui fazemos isso e logo em seguida guardamos o resultado na nossa esturua de heap.

In [4]:
chars_frequency = {}
for character in text:
    if not character in chars_frequency:
        chars_frequency[character] = 0
    chars_frequency[character] += 1

print('\nFrequência dos caracteres:\n')
print({k: v for k, v in sorted(chars_frequency.items(), key=lambda item: item[1])})

heap = []

for key in chars_frequency:
    node = HeapNode(key, chars_frequency[key])
    heapq.heappush(heap, node)


Frequência dos caracteres:

{'N': 1, 'A': 1, 'á': 2, 'T': 3, ',': 3, 'S': 3, 'q': 4, 'L': 4, 'É': 4, 'j': 5, 'x': 5, 'ã': 8, 'ó': 8, '?': 8, 'E': 9, 'z': 9, 'h': 10, 'M': 10, 'Q': 12, 'g': 12, 'f': 14, 'V': 14, 'ê': 15, 'p': 16, '.': 18, 't': 23, 'v': 25, 'b': 33, 'l': 35, 'd': 36, 'n': 39, 'c': 40, '\n': 46, 'u': 49, 'i': 65, 'r': 68, 'm': 68, 's': 72, 'o': 108, 'a': 111, 'e': 139, ' ': 235}


### 1.4 Codificação

Agora construímos nossa árvore com os caracteres, de forma que os nós mais próximos da raiz são aqueles que representam os caracteres com maior frequência. Isso fica especialmente fácil por termos utilizado uma heap, já guardando os nós em uma ordem conveniente.

Esse processo é feito da seguinte forma:

<b>1º</b> Selecionar os dois nós com menor valor de frequência e unir ambos em um nó pai cujo valor é a soma dos dois.

<b>2º</b> Fazer isso repetidamente até termos uma única árvore.

<b>3º</b> A partir da raiz da árvore visitar todos os filhos, para cada um deles atribuir 0s pra toda aresta a esquerda e 1s para toda aresta a direita.


<b>Abaixo segue um GIF ilustrando o processo: </b>

![huf_gif](huffman.gif)
<br>
<b>Um outro exemplo: </b>
<br>
![tree2_1](tree2_1.png)
![tree2_2](tree2_2.png)


Execução dos dois primeiros passos:

In [5]:
while(len(heap)>1):
    node1 = heapq.heappop(heap)
    node2 = heapq.heappop(heap)

    merged = HeapNode(None, node1.freq + node2.freq)
    merged.left = node1
    merged.right = node2

    heapq.heappush(heap, merged)

<div style="font-size:16px">
Estando com a árvore montada, podemos construir os códigos para cada caráter, que é o nosso passo três. Aqui fazemos isso recursivamente. 

Enquanto numeramos as arestas da árvore, os códigos de cada caráter são os números que levam da raiz até ele. 

Na última foto por exemplo o código para o caráter "U" seria "001", pois para chegar a ele da raiz vamos duas vezes para esquerda e por fim para direita, resultando em dois 0s e um 1.

Além disso, construímos também um outro dicionário, que chamamos de "reverse_mapping". Esse dicionário irá conter a correspondência CODIGO:CARACTER, os códigos sendo as chaves do dicionário. Fazemos isso porque mais tarde, na hora de descodificar uma string de bits, fica mais fácil achar o caráter que corresponde a um código com o dicionário dessa forma
</div>

In [6]:

reverse_mapping = {}
codes = {}

def build_huffman_code(root, current_code):
    if(root == None):
        return

    if(root.char != None):
        codes[root.char] = current_code
        reverse_mapping[current_code] = root.char
        return

    build_huffman_code(root.left, current_code + "0") # Cada vez que vamos para esquerda, atribuimos mais um 0  
    build_huffman_code(root.right, current_code + "1") # Cada vez que vamos para direita, atribuimos um 1
    
    
build_huffman_code(heapq.heappop(heap), "")
print("\nCódigos:\n")
print(codes)


Códigos:

{'i': '0000', 'r': '0001', 'm': '0010', 'b': '00110', 'l': '00111', 'e': '010', 's': '0110', '.': '011100', 'L': '01110100', 'j': '01110101', 'z': '0111011', 'd': '01111', 'n': '10000', 'c': '10001', 'M': '1001000', 'h': '1001001', 'x': '10010100', 'T': '100101010', 'S': '100101011', 'g': '1001011', '\n': '10011', 't': '101000', 'v': '101001', 'u': '10101', 'o': '1011', 'a': '1100', 'Q': '1101000', 'f': '1101001', 'V': '1101010', 'ê': '1101011', ',': '110110000', 'q': '110110001', 'ó': '11011001', 'p': '1101101', 'ã': '11011100', 'É': '110111010', 'á': '1101110110', 'N': '11011101110', 'A': '11011101111', '?': '11011110', 'E': '11011111', ' ': '111'}


Tendo os códigos para cada caracter, podemos juntar tudo e criar o código que representa o texto inteiro

In [7]:
encoded_text = ""
for character in text:
    encoded_text += codes[character]
print(encoded_text)

1101000101011100100000111110111110101010111110100001011110100101001110101101110011110111110110110110101000011011111101000010101011110011001000000111010110111001111011101110110111001011111011000001000010100010111111010010100001100101110111000010010011100100111101110111111011010101000011000110111011110100110010011101011011100111001000000010000100100111001110011010111000111001110101000010001101101101010001100100111101111100101111010001010111001110011010111000111001111101100011010101011110100000010100010010100111001000010101010110111101100111100100110110110111010101011111101001010100011001001101110011100100011000110111101101101111010000101010101101110100110101000110111001011111110000110010000110100010110110100111001010101010101111101111100110010001011101100101111000011011100101111101111010101011111000101000011010001011100111101111110101111110010001100100101000001111101100011010101011110000110110010110111100011001001010100101111000010101101101111010001101110010111111101101010000110100010111

### 1.5 Construção dos bytes

Se o número de bits não for múltiplo de 8 temos que adicionar alguns bits extras. Isso porque na hora de ler o arquivo para decodifica-lo leremos os bytes do arquivo, não os bits, então utilizamos essa estratégia para não bagunçar os códigos de cada carácter na hora da leitura para descompressão.

Após fazer isso, guardamos no inicio do novo código formado a informação de quantos bits extras existem, para que possamos desconsidera-los na hora da tradução.

In [8]:
extra_padding = 8 - len(encoded_text) % 8
for i in range(extra_padding):
    encoded_text += "0"

padded_info = "{0:08b}".format(extra_padding)
encoded_text = padded_info + encoded_text

### 1.6 Armazenamento



In [9]:
compressed_path=f'{original_filename}_compressed.bin'
with open(compressed_path, 'wb') as output:

    btarray = bytearray()
    for i in range(0, len(encoded_text), 8):
        byte = encoded_text[i:i+8]
        btarray.append(int(byte, 2))        
        
    output.write(bytes(btarray))
print(f'Arquivo salvo como: {compressed_path}')

Arquivo salvo como: arquivos_exemplo/musica_vai_lembrar_de_mim_compressed.bin


Por fim, verificamos a eficiência da compressão 

In [10]:
original_file_size = os.path.getsize(original_path)
compressed_file_size = os.path.getsize(compressed_path)
size_reduction =((original_file_size-compressed_file_size)/original_file_size)*100

display(HTML(f'<b>Nome do arquivo original: </b>{original_path}<br>'+
             f'<b>Nome do arquivo comprimido: </b>{compressed_path}<br>'+
             f'<b>Tamanho do arquivo original: </b>{original_file_size} bytes<br>'+
             f'<b>Tamanho do arquivo comprimido: </b>{compressed_file_size} bytes<br>'+
             f'<br>O algoritmo conseguiu reduzir o tamanho do arquivo para cerca de  <b>{round(size_reduction,0)}%</b> do seu tamanho original<br>'+
             
             f'<br><img src="file.png" style="display:inline;margin:1px" width=100px height=100px/>'+
             f'<img src="arrow.png" style="display:inline;margin:1px" width=40px height=40px/>'+
             f'<img src="file.png" style="display:inline;margin:1px" width={size_reduction}px height={size_reduction}px/>'
            ))


# PT 2. Descodificação

### 2.1 Leitura do arquivo 
O primeiro passo para iniciar uma descodificação é ler o arquivo binário que contem o conteúdo comprimido. Realizamos a leitura byte a byte 

In [11]:
compressed_filename, compressed_file_extension = os.path.splitext(compressed_path)

with open(compressed_path, 'rb') as file:
    bits_string = ""

    byte = file.read(1) # Leitura do primeiro byte
    while(len(byte) > 0):
        byte = ord(byte) # Conversão do byte para uma representação inteira
        bits = bin(byte)[2:].rjust(8, '0') #Eliminamos o prefixo "0b" que precede os bits e convertemos o byte para uma string de bits
        bits_string += bits
        byte = file.read(1) # Leitura dos proximos bytes

### 2.2 Remoção dos bits extras 

No passo de compressão adicionamos alguns bits extras ao código formado quando é o caso de o número de bits não ser múltiplo de 8. Além disso, guardamos a informação de quantos bits extras existem logo no primeiro byte. Logo, agora basta lermos essa informação para sabermos quantos bits retirar. 

In [12]:
padded_info = bits_string[:8]
extra_padding = int(padded_info, 2)

bits_string = bits_string[8:] 
encoded_text = bits_string[:-1*extra_padding]

### 2.3 Tradução do código para texto 

Agora que temos a string com todos os bits do nosso conteúdo podemos realizar a tradução e obter o texto original. Para isso utilizamos o nosso dicionario "reverse_mapping", que foi construído tendo os códigos como chave e os caracteres como valor. 

In [13]:
current_code = ""
decompressed_text = ""

for bit in encoded_text:
    current_code += bit
    if(current_code in reverse_mapping):
        character = reverse_mapping[current_code]
        decompressed_text += character
        current_code = ""
display(HTML('<b>Texto resultante: </b>'))
print(decompressed_text)

Quando eu te vejo
Espero teu beijo
Não sinto vergonha
Apenas desejo
Minha boca encosta
Em tua boca que treme
Meus olhos eu fecho
Mas os teus estão abertos
Tudo bem se não deu certo
Eu achei que nós chegamos tão perto
Mas agora, com certeza eu enxergo
Que no fim eu amei por nós dois
Esse foi um beijo de despedida
Que se dá uma vez só na vida
Que explica tudo sem brigas
E clareia o mais escuro dos dias
Tudo bem se não deu certo
Eu achei que nós chegamos tão perto
Mas agora, com certeza eu enxergo
Que no fim eu amei por nós dois
Mas você lembra? Você vai lembrar de mim
Que o nosso amor valeu a pena
Lembra? É o nosso final feliz
Você vai lembrar... Vai lembrar... Sim
Você vai lembrar de mim
Esse foi um beijo de despedida
Que se dá uma vez só na vida
Que explica tudo sem brigas
E clareia o mais escuro dos dias
Tudo bem se não deu certo
Eu achei que nós chegamos tão perto
Mas agora, com certeza eu enxergo
Que no fim eu amei por nós dois
Mas você lembra? Você vai lembrar de mim
Que o nosso am

### 2.4 Armazenamento 


In [14]:
decompressed_file_path=f'{original_filename}_decompressed.txt'
with open(decompressed_file_path, 'w') as output:
    output.write(decompressed_text)
print(f'O arquivo descompactado foi salvo como {decompressed_file_path}')

O arquivo descompactado foi salvo como arquivos_exemplo/musica_vai_lembrar_de_mim_decompressed.txt


# PT 3. Teste

Por fim, é possível comparar o arquivo original com o arquivo descomprimido para averiguar se os dois são de fato iguais

In [15]:
if filecmp.cmp(original_path, decompressed_file_path):
    print("O conteúdo do arquivo original e do arquivo construído pela descodificação é o mesmo")
else:
    print("Algo deu errado")

O conteúdo do arquivo original e do arquivo construído pela descodificação é o mesmo
